In [211]:
!pip install lightgbm

     ---------------------------------------- 1.3/1.3 MB 165.3 kB/s eta 0:00:00


In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored

# Print bold and colored text
from IPython.display import HTML

# Create HTML code to display red and bold text
red_bold_text = '<span style="color:red; font-weight:bold;">This is red and bold text</span>'

# Display red and bold text in the notebook cell
HTML(red_bold_text)

In [305]:
df = pd.read_csv('marketing_campaign_data.csv')
# Assume that maximum date is Year 2014

df['Age'] = 2014 - df['Year_Birth']
df['enrolled_day'] = (pd.to_datetime('6/29/2014') - pd.to_datetime(df['Dt_Customer'])).dt.days

df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2240.0,5592.159821,3246.662198,0.0,2828.25,5458.5,8427.75,11191.0
Year_Birth,2240.0,1968.805804,11.984069,1893.0,1959.00,1970.0,1977.00,1996.0
Income,2216.0,52247.251354,25173.076661,1730.0,35303.00,51381.5,68522.00,666666.0
Kidhome,2240.0,0.444196,0.538398,0.0,0.00,0.0,1.00,2.0
Teenhome,2240.0,0.506250,0.544538,0.0,0.00,0.0,1.00,2.0
Recency,2240.0,49.109375,28.962453,0.0,24.00,49.0,74.00,99.0
MntWines,2240.0,303.935714,336.597393,0.0,23.75,173.5,504.25,1493.0
MntFruits,2240.0,26.302232,39.773434,0.0,1.00,8.0,33.00,199.0
MntMeatProducts,2240.0,166.950000,225.715373,0.0,16.00,67.0,232.00,1725.0
MntFishProducts,2240.0,37.525446,54.628979,0.0,3.00,12.0,50.00,259.0


In [6]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,9/4/2012,58,635,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,3/8/2014,38,11,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,8/21/2013,26,426,...,2,10,4,0,0,0,0,0,0,0
3,6182,1984,Graduation,Together,26646.0,1,0,2/10/2014,26,11,...,0,4,6,0,0,0,0,0,0,0
4,5324,1981,PhD,Married,58293.0,1,0,1/19/2014,94,173,...,3,6,5,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

# Feature

1. Flag Campaign Acceptance >> รับข้อเสนอแคมเปญที่ n

    - AcceptedCmp1
    - AcceptedCmp2
    - AcceptedCmp3
    - AcceptedCmp4
    - AcceptedCmp5
    - **Response** (Target) 
    
    
2. Demographic Data

    - Education
    - Marital
    - Kidhome
    - Teenhome
    - Income
    
    
3. RFM Data
    
    3.1. Recency
    
        - Recency
        
    3.2. Frequency
   
        - NumDealsPurchases
        - NumCatalogPurchases
        - NumStorePurchases
        - NumWebPurchases
        - NumWebVisitsPurchases
    
    3.3. Monetary
        
        - MntFishProducts
        - MntMeatProducts
        - MntFruits
        - MntSweetProducts
        - MntWines
        - MntGoldProds
        
4. Others

    - Complain
    - DtCustomer

# Create Utility Function

In [132]:
def check_is_number(df, col):
    
    return 'float' in str(df[col].dtypes) or 'int' in str(df[col].dtypes)

def create_summary_table(df, col):
    
    dff = df.copy()
    
    isNum = check_is_number(dff, col)

    if isNum:
        try:
            try:
                dff[f'{col}_bin'] = pd.qcut(dff[col], q = 8)
            except:
                try:
                    dff[f'{col}_bin'] = pd.qcut(dff[col], q = 5)
                except:
                    dff[f'{col}_bin'] = pd.qcut(dff[col], q = 3)
            tmp = dff.groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : 'Total'})
            tmp_res_0 = dff[dff['Response'] == 0].groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : '# Response = 0'})
            tmp_res_1 = dff[dff['Response'] == 1].groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : '# Response = 1'})

            result = pd.merge(tmp, tmp_res_0, how = 'left', on = f'{col}_bin')
            result = pd.merge(result, tmp_res_1, how = 'left', on = f'{col}_bin')
            result['%Total'] = np.round(tmp['Total']/tmp['Total'].sum(), 2)
            result['%Event Rate'] = np.round(result['# Response = 1']/result['Total'], 2)
            
        except:
            dff[f'{col}_bin'] = dff[col]
        
            tmp = dff.groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : 'Total'})
            tmp_res_0 = dff[dff['Response'] == 0].groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : '# Response = 0'})
            tmp_res_1 = dff[dff['Response'] == 1].groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : '# Response = 1'})

            result = pd.merge(tmp, tmp_res_0, how = 'left', on = f'{col}_bin')
            result = pd.merge(result, tmp_res_1, how = 'left', on = f'{col}_bin')
            result['%Total'] = np.round(tmp['Total']/tmp['Total'].sum(), 2)
            result['%Event Rate'] = np.round(result['# Response = 1']/result['Total'], 2)
    
    else:
        dff[f'{col}_bin'] = dff[col]
        
        tmp = dff.groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : 'Total'})
        tmp_res_0 = dff[dff['Response'] == 0].groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : '# Response = 0'})
        tmp_res_1 = dff[dff['Response'] == 1].groupby([f'{col}_bin'])[['ID']].count().rename(columns = {'ID' : '# Response = 1'})
        
        result = pd.merge(tmp, tmp_res_0, how = 'left', on = f'{col}_bin')
        result = pd.merge(result, tmp_res_1, how = 'left', on = f'{col}_bin')
        result['%Total'] = np.round(tmp['Total']/tmp['Total'].sum(), 2)
        result['%Event Rate'] = np.round(result['# Response = 1']/result['Total'], 2)
        result = result.sort_values(by = '%Event Rate')
    return result.style.bar(subset = '%Event Rate', color = '#F2DA7A')

create_summary_table(df, 'AcceptedCmp1')


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
AcceptedCmp1_bin,,,,,
0,2096,1841,255,0.940000,0.120000
1,144,65,79,0.060000,0.550000


## 1. Campaign Acceptance Data

In [155]:
feature = sorted([i for i in df.columns if i.startswith('Accepted')])
print(feature)
print('==============\n')
for col in feature:
    text = f'<span style="color:red; font-weight:bold;">{col}</span>'
    display(HTML(text))
    display(create_summary_table(df, col))

['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']



,Total,# Response = 0,# Response = 1,%Total,%Event Rate
AcceptedCmp1_bin,,,,,
0,2096,1841,255,0.940000,0.120000
1,144,65,79,0.060000,0.550000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
AcceptedCmp2_bin,,,,,
0,2210,1896,314,0.990000,0.140000
1,30,10,20,0.010000,0.670000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
AcceptedCmp3_bin,,,,,
0,2077,1820,257,0.930000,0.120000
1,163,86,77,0.070000,0.470000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
AcceptedCmp4_bin,,,,,
0,2073,1801,272,0.930000,0.130000
1,167,105,62,0.070000,0.370000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
AcceptedCmp5_bin,,,,,
0,2077,1835,242,0.930000,0.120000
1,163,71,92,0.070000,0.560000


## 2. Demographic Data

In [161]:
df['Year_Birth']

# Assume that maximum date is Year 2014

df['Age'] = 2014 - df['Year_Birth']
df['Age']

0       57
1       60
2       49
3       30
4       33
        ..
2235    47
2236    68
2237    33
2238    58
2239    60
Name: Age, Length: 2240, dtype: int64

In [174]:
df['Income'] = df['Income'].fillna(-9999999) # Fill na with Special Value

In [175]:
feature = ['Education', 'Marital_Status', 'Kidhome', 'Teenhome', 'Income', 'Age']
print(feature)
print('==============\n')
for col in feature:
    text = f'<span style="color:red; font-weight:bold;">{col}</span>'
    display(HTML(text))
    display(create_summary_table(df, col))

['Education', 'Marital_Status', 'Kidhome', 'Teenhome', 'Income', 'Age']



,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Education_bin,,,,,
Basic,54,52,2,0.020000,0.040000
2n Cycle,203,181,22,0.090000,0.110000
Graduation,1127,975,152,0.500000,0.130000
Master,370,313,57,0.170000,0.150000
PhD,486,385,101,0.220000,0.210000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Marital_Status_bin,,,,,
Together,580,520,60,0.260000,0.100000
Married,864,766,98,0.390000,0.110000
Divorced,232,184,48,0.100000,0.210000
Single,480,374,106,0.210000,0.220000
Widow,77,58,19,0.030000,0.250000
Alone,3,2,1,0.000000,0.330000
Absurd,2,1,1,0.000000,0.500000
YOLO,2,1,1,0.000000,0.500000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Kidhome_bin,,,,,
0,1293,1071,222,0.580000,0.170000
1,899,789,110,0.400000,0.120000
2,48,46,2,0.020000,0.040000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Teenhome_bin,,,,,
0,1158,921,237,0.520000,0.200000
1,1030,938,92,0.460000,0.090000
2,52,47,5,0.020000,0.100000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Income_bin,,,,,
"(-9999999.001, 25540.5]",280,251,29,0.120000,0.100000
"(25540.5, 34722.0]",280,254,26,0.120000,0.090000
"(34722.0, 42419.25]",280,239,41,0.120000,0.150000
"(42419.25, 51075.0]",280,248,32,0.120000,0.110000
"(51075.0, 60055.5]",280,255,25,0.120000,0.090000
"(60055.5, 68289.75]",280,248,32,0.120000,0.110000
"(68289.75, 77623.25]",280,237,43,0.120000,0.150000
"(77623.25, 666666.0]",280,174,106,0.120000,0.380000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Age_bin,,,,,
"(17.999, 31.0]",301,243,58,0.130000,0.190000
"(31.0, 37.0]",305,262,43,0.140000,0.140000
"(37.0, 40.0]",241,211,30,0.110000,0.120000
"(40.0, 44.0]",317,262,55,0.140000,0.170000
"(44.0, 49.0]",290,255,35,0.130000,0.120000
"(49.0, 55.0]",267,233,34,0.120000,0.130000
"(55.0, 60.0]",250,214,36,0.110000,0.140000
"(60.0, 121.0]",269,226,43,0.120000,0.160000


## 3. RFM Data

In [157]:
feature = [i for i in df.columns if i == 'Recency' or i.startswith('Mnt') or i.startswith('Num')]
print(feature)
print('==============\n')
for col in feature:
    text = f'<span style="color:red; font-weight:bold;">{col}</span>'
    display(HTML(text))
    display(create_summary_table(df, col))
    print('\n')

['Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']



,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Recency_bin,,,,,
"(-0.001, 11.0]",285,199,86,0.130000,0.300000
"(11.0, 24.0]",282,218,64,0.130000,0.230000
"(24.0, 37.0]",295,247,48,0.130000,0.160000
"(37.0, 49.0]",260,229,31,0.120000,0.120000
"(49.0, 62.0]",296,256,40,0.130000,0.140000
"(62.0, 74.0]",271,247,24,0.120000,0.090000
"(74.0, 87.0]",297,272,25,0.130000,0.080000
"(87.0, 99.0]",254,238,16,0.110000,0.060000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
MntWines_bin,,,,,
"(-0.001, 8.0]",284,259,25,0.130000,0.090000
"(8.0, 23.75]",276,250,26,0.120000,0.090000
"(23.75, 67.0]",281,259,22,0.130000,0.080000
"(67.0, 173.5]",279,239,40,0.120000,0.140000
"(173.5, 320.375]",280,246,34,0.120000,0.120000
"(320.375, 504.25]",280,243,37,0.120000,0.130000
"(504.25, 752.125]",280,235,45,0.120000,0.160000
"(752.125, 1493.0]",280,175,105,0.120000,0.380000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
MntFruits_bin,,,,,
"(-0.001, 1.0]",562,513,49,0.250000,0.090000
"(1.0, 4.0]",340,300,40,0.150000,0.120000
"(4.0, 15.0]",467,414,53,0.210000,0.110000
"(15.0, 44.0]",431,338,93,0.190000,0.220000
"(44.0, 199.0]",440,341,99,0.200000,0.220000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
MntMeatProducts_bin,,,,,
"(-0.001, 8.0]",307,295,12,0.140000,0.040000
"(8.0, 16.0]",285,260,25,0.130000,0.090000
"(16.0, 30.0]",263,234,29,0.120000,0.110000
"(30.0, 67.0]",267,224,43,0.120000,0.160000
"(67.0, 124.0]",281,244,37,0.130000,0.130000
"(124.0, 232.0]",279,244,35,0.120000,0.130000
"(232.0, 431.125]",278,225,53,0.120000,0.190000
"(431.125, 1725.0]",280,180,100,0.120000,0.360000


...

,Total,# Response = 0,# Response = 1,%Total,%Event Rate
NumDealsPurchases_bin,,,,,
"(-0.001, 1.0]",1016,840,176,0.450000,0.170000
"(1.0, 2.0]",497,449,48,0.220000,0.100000
"(2.0, 15.0]",727,617,110,0.320000,0.150000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
NumWebPurchases_bin,,,,,
"(-0.001, 1.0]",403,381,22,0.180000,0.050000
"(1.0, 2.0]",373,343,30,0.170000,0.080000
"(2.0, 3.0]",336,282,54,0.150000,0.160000
"(3.0, 4.0]",280,238,42,0.120000,0.150000
"(4.0, 5.0]",220,162,58,0.100000,0.260000
"(5.0, 6.0]",205,176,29,0.090000,0.140000
"(6.0, 7.0]",155,110,45,0.070000,0.290000
"(7.0, 27.0]",268,214,54,0.120000,0.200000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
NumCatalogPurchases_bin,,,,,
"(-0.001, 1.0]",1083,990,93,0.480000,0.090000
"(1.0, 3.0]",460,395,65,0.210000,0.140000
"(3.0, 28.0]",697,521,176,0.310000,0.250000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
NumStorePurchases_bin,,,,,
"(-0.001, 3.0]",735,657,78,0.330000,0.110000
"(3.0, 4.0]",323,282,41,0.140000,0.130000
"(4.0, 6.0]",390,309,81,0.170000,0.210000
"(6.0, 9.0]",398,319,79,0.180000,0.200000
"(9.0, 13.0]",394,339,55,0.180000,0.140000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
NumWebVisitsMonth_bin,,,,,
"(-0.001, 3.0]",571,468,103,0.250000,0.180000
"(3.0, 5.0]",499,448,51,0.220000,0.100000
"(5.0, 6.0]",340,292,48,0.150000,0.140000
"(6.0, 7.0]",393,348,45,0.180000,0.110000
"(7.0, 20.0]",437,350,87,0.200000,0.200000


# 4. Other Data

In [116]:
feature = ['Complain', 'Dt_Customer']
df[feature]

,Complain,Dt_Customer
0,0,9/4/2012
1,0,3/8/2014
2,0,8/21/2013
3,0,2/10/2014
4,0,1/19/2014
...,...,...
2235,0,6/13/2013
2236,0,6/10/2014
2237,0,1/25/2014
2238,0,1/24/2014


We need to calculate how long the customer have been enrolled in our company but..

- We don't know which date is today (Today = Day at launching the latest campaign)

Solution : Assume Max Enroll Date of Customers and add 1 month as **TODAY**

In [120]:
print(pd.to_datetime(df['Dt_Customer']).max())

2014-06-29 00:00:00


- So we assume 6/29/2024 as a TODAY

In [122]:
df['enrolled_day'] = (pd.to_datetime('6/29/2014') - pd.to_datetime(df['Dt_Customer'])).dt.days

0       663
1       113
2       312
3       139
4       161
       ... 
2235    381
2236     19
2237    155
2238    156
2239    622
Name: enrolled_day, Length: 2240, dtype: int64

In [158]:
feature = ['Complain', 'enrolled_day']

for col in feature:
    text = f'<span style="color:red; font-weight:bold;">{col}</span>'
    display(HTML(text))
    display(create_summary_table(df, col))

,Total,# Response = 0,# Response = 1,%Total,%Event Rate
Complain_bin,,,,,
0,2219,1888,331,0.990000,0.150000
1,21,18,3,0.010000,0.140000


,Total,# Response = 0,# Response = 1,%Total,%Event Rate
enrolled_day_bin,,,,,
"(-0.001, 89.875]",280,261,19,0.120000,0.070000
"(89.875, 180.75]",280,251,29,0.120000,0.100000
"(180.75, 268.0]",281,254,27,0.130000,0.100000
"(268.0, 355.5]",279,252,27,0.120000,0.100000
"(355.5, 442.0]",284,246,38,0.130000,0.130000
"(442.0, 529.0]",282,233,49,0.130000,0.170000
"(529.0, 617.0]",275,216,59,0.120000,0.210000
"(617.0, 699.0]",279,193,86,0.120000,0.310000


In [284]:
df_cond = df[(df['AcceptedCmp1'] == 1) |
  (df['AcceptedCmp2'] == 1) |
  (df['AcceptedCmp3'] == 1) |
  (df['AcceptedCmp4'] == 1) | 
  (df['AcceptedCmp5'] == 1) |
  (df['Education'].isin(['Master', 'PhD'])) |
  (df['Marital_Status'].isin(['Divorced', 'Single', 'Widow', 'Alone', 'Absurd', 'YOLO'])) |
  (df['Kidhome'] == 0) |
  (df['Teenhome'] == 0) |
  (df['Income'] > 68289.75) |
  (df['Recency'] <= 11) |
  (df['MntWines'] > 504.25) |
  (df['MntFruits'] > 15) |
  (df['MntMeatProducts'] > 431.25) |
  (df['MntFishProducts'] > 20) |
  (df['MntSweetProducts'] > 14) |
  (df['MntGoldProds'] > 15) |
  (df['NumWebPurchases'] > 4) |
  (df['NumCatalogPurchases'] > 3) | 
  (df['NumStorePurchases'] > 4) |
  (df['enrolled_day'] >= 529)]

event_old = round(df[df['Response'] == 1].shape[0]/df.shape[0]*100,2)
event_new = round(df_cond[df_cond['Response'] == 1].shape[0]/df_cond.shape[0]*100, 2)

print(f'Count Old : {df.shape[0]}')
print(f'Count New : {df_cond.shape[0]}')
print('=====================')
print(f'Event Rate Old : {event_old} %')
print(f'Event Rate New : {event_new} %')

Count Old : 2240
Count New : 2193
Event Rate Old : 14.91 %
Event Rate New : 15.18 %


# Hypothesis testing

In [165]:
cat_cols = ['Education', 'Marital_Status', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', ]
num_cols = ['Income', 'Kidhome',
       'Teenhome', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'enrolled_day', 'Age']
target = 'Response'

In [185]:
from scipy.stats import chi2_contingency, ttest_ind

fts = []
ps = []
# Create a contingency table
for col in num_cols:
    
    group0 = df[df[target] == 0][col]  # Sample data for Group 1
    group1 = df[df[target] == 1][col]  # Sample data for Group 2

    # Perform a two-sample t-test
    t_statistic, p_value = ttest_ind(group0, group1)
    fts.append(col)
    ps.append(p_value)
    
num_result = pd.DataFrame({
    'Feature' : fts,
    'P-value' : np.round(ps,6)
}).sort_values(by = 'P-value')

text = f'<span style="color:red; font-weight:bold;">Numerical Features</span>'
display(HTML(text))
display(num_result)

,Feature,P-value
8,MntSweetProducts,0.000000
2,Teenhome,0.000000
3,Recency,0.000000
4,MntWines,0.000000
5,MntFruits,0.000000
6,MntMeatProducts,0.000000
7,MntFishProducts,0.000000
15,enrolled_day,0.000000
9,MntGoldProds,0.000000
11,NumWebPurchases,0.000000


In [186]:
fts = []
ps = []

for col in cat_cols:
    
    contingency_table = pd.crosstab(df[col], df[target])

    # Perform chi-squared test for independence
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    fts.append(col)
    ps.append(p_value)
    
cat_result = pd.DataFrame({
    'Feature' : fts,
    'P-value' : np.round(ps,6)
}).sort_values(by = 'P-value')

text = f'<span style="color:red; font-weight:bold;">Categorical Features</span>'
display(HTML(text))
display(cat_result)

,Feature,P-value
1,Marital_Status,0.000000
2,AcceptedCmp3,0.000000
3,AcceptedCmp4,0.000000
4,AcceptedCmp5,0.000000
5,AcceptedCmp1,0.000000
6,AcceptedCmp2,0.000000
0,Education,0.000122
7,Complain,1.000000


# ML Model Approach

**Challenge**
- Assume that this campaign will also run onwards recently
- We would try to launch this campaign into the rest of leads 
- But the challenge is we try to optimize budget cost **i.e.** effectively grab only potential leads (dont want to waste money on unaccepting leads)


**Solution**
- Creating classification model using historical data to determine which leads would accurately accept our campaign


In [189]:
feature = num_cols + cat_cols
print(feature)
print(target)

['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'enrolled_day', 'Age', 'Education', 'Marital_Status', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain']
Response


In [212]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

dff = df.copy()

label_encoders = {}

X_train, X_test, y_train, y_test = train_test_split(df[feature], df[target], test_size = 0.3, random_state = 42)

for col in cat_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

In [213]:
from sklearn.tree import DecisionTreeClassifier

clf = lgb.LGBMClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)[:,1]
y_pred

[LightGBM] [Info] Number of positive: 239, number of negative: 1329
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000229 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1762
[LightGBM] [Info] Number of data points in the train set: 1568, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.152423 -> initscore=-1.715719
[LightGBM] [Info] Start training from score -1.715719


array([1.36687134e-02, 6.10266861e-02, 2.62220932e-03, 2.30594795e-01,
       7.43218856e-03, 5.40837685e-04, 5.26515272e-05, 3.94380989e-05,
       2.33439419e-02, 4.92203246e-05, 2.95339988e-02, 2.16318482e-01,
       3.33174448e-02, 5.90107770e-01, 9.96551500e-04, 2.21115531e-03,
       2.13492657e-02, 3.38445876e-01, 6.15491391e-04, 2.39927038e-03,
       9.19219093e-01, 5.14537556e-02, 4.54620347e-02, 9.14743458e-01,
       6.76524700e-03, 3.37423149e-01, 6.91368935e-03, 4.15589088e-01,
       3.78783475e-01, 8.41288457e-04, 4.93771747e-02, 3.65582104e-03,
       4.35096943e-03, 1.55156441e-04, 8.64496925e-01, 1.19303186e-02,
       8.48457520e-01, 3.35794415e-04, 1.44694564e-03, 2.36789623e-03,
       2.52311757e-03, 1.30535441e-03, 7.73031404e-04, 3.51414593e-02,
       6.92895174e-03, 6.07887702e-02, 4.29578503e-03, 2.34945559e-04,
       1.39247839e-01, 1.39901221e-02, 4.80702776e-05, 1.47542742e-02,
       9.63152215e-01, 1.63956994e-02, 1.57592096e-04, 1.35497021e-03,
      

In [223]:
pd.DataFrame({
    'Feature' : feature,
    'Importance' : clf.feature_importances_/clf.feature_importances_.sum()
}).sort_values(by = 'Importance', ascending = False)

,Feature,Importance
6,MntMeatProducts,0.090667
3,Recency,0.088000
15,enrolled_day,0.083667
4,MntWines,0.077000
0,Income,0.069333
9,MntGoldProds,0.064667
16,Age,0.063000
7,MntFishProducts,0.057000
13,NumStorePurchases,0.049667
8,MntSweetProducts,0.048000


In [303]:
from sklearn.metrics import roc_auc_score

y_train_prob = clf.predict_proba(X_train)[:,1]
y_test_prob = clf.predict_proba(X_test)[:,1]

tmp1 = pd.DataFrame({
    'type' : 'y_train',
    'predict_prob' : y_train_prob,
    'Response' : y_train
})
tmp2 = pd.DataFrame({
    'type' : 'y_test',
    'predict_prob' : y_test_prob,
    'Response' : y_test
})
tmp = pd.concat([tmp1,tmp2], ignore_index = True)

_, bins = pd.qcut(tmp[tmp['type'] == 'y_train']['predict_prob'], q= 20, retbins = True)
tmp['predict_prob_bin'] = pd.cut(tmp['predict_prob'], bins)

cnt_train = tmp[tmp['type'] == 'y_train'].groupby(['predict_prob_bin'])[['type']].count().rename(columns = {'type' : 'Total'})
cnt_non_accept_train = tmp[(tmp['type'] == 'y_train') & (tmp['Response'] == 0)]\
                        .groupby(['predict_prob_bin'])[['type']].count().rename(columns = {'type' : '#Response = 0'})
cnt_accept_train = tmp[(tmp['type'] == 'y_train') & (tmp['Response'] == 1)]\
                        .groupby(['predict_prob_bin'])[['type']].count().rename(columns = {'type' : '#Response = 1'})

cnt_test = tmp[tmp['type'] == 'y_test'].groupby(['predict_prob_bin'])[['type']].count().rename(columns = {'type' : '(Test) Total'})
cnt_non_accept_test = tmp[(tmp['type'] == 'y_test') & (tmp['Response'] == 0)]\
                        .groupby(['predict_prob_bin'])[['type']].count().rename(columns = {'type' : '(Test) #Response = 0'})
cnt_accept_test = tmp[(tmp['type'] == 'y_test') & (tmp['Response'] == 1)]\
                        .groupby(['predict_prob_bin'])[['type']].count().rename(columns = {'type' : '(Test) #Response = 1'})

result = pd.merge(cnt_train, cnt_non_accept_train, how = 'left', on = 'predict_prob_bin')
result = pd.merge(result, cnt_accept_train, how = 'left', on ='predict_prob_bin')
result['(Train) %Event Rate'] = np.round(result['#Response = 1']/result['Total'],2)
result = pd.merge(result, cnt_test, how = 'left', on = 'predict_prob_bin')
result = pd.merge(result, cnt_non_accept_test, how = 'left', on = 'predict_prob_bin')
result = pd.merge(result, cnt_accept_test, how = 'left', on ='predict_prob_bin')
result['(Test) %Event Rate'] = np.round(result['(Test) #Response = 1']/result['(Test) Total'],2)

def gini_coefficient(y_actual, y_pred_prob):
    """
    Calculate the Gini coefficient given the actual and predicted probability values.
    """
    # Calculate AUC
    auc = roc_auc_score(y_actual, y_pred_prob)
    
    # Calculate Gini coefficient
    gini = 2 * auc - 1
    return gini

gini_train = round(gini_coefficient(y_train, y_train_prob), 3)
gini_test = round(gini_coefficient(y_test, y_test_prob), 3)

print(f'Gini train : {gini_train}')
print(f'Gini test : {gini_test}')
result

Gini train : 1.0
Gini test : 0.737


,Total,#Response = 0,#Response = 1,(Train) %Event Rate,(Test) Total,(Test) #Response = 0,(Test) #Response = 1,(Test) %Event Rate
predict_prob_bin,,,,,,,,
"(2.33e-05, 8.85e-05]",78,78,0,0.00,40,40,0,0.00
"(8.85e-05, 0.000143]",78,78,0,0.00,25,25,0,0.00
"(0.000143, 0.000199]",79,79,0,0.00,27,27,0,0.00
"(0.000199, 0.000302]",78,78,0,0.00,29,29,0,0.00
"(0.000302, 0.00043]",78,78,0,0.00,39,38,1,0.03
"(0.00043, 0.000592]",79,79,0,0.00,30,28,2,0.07
"(0.000592, 0.000812]",78,78,0,0.00,35,34,1,0.03
"(0.000812, 0.00118]",78,78,0,0.00,31,30,1,0.03
"(0.00118, 0.00165]",79,79,0,0.00,32,32,0,0.00


# Prescriptive Analysis

## Cufoff Selection

- Normally we use 0.5 as a cutoff to distinguish acceptance and non-acceptance
- But, we will use an optimal number that come from optimization to maximize our profit

**Proposal**
1. We assume that cost of campaign per customer is various with launched channel i.e.
- Online cost is 1000 THB (less effort)
- Offline cost (at shop) is 1500 THB (more effort, higher cost)

Then : We would randomly assigned channel of campaign with a proportion of **Online : Offline = 65 : 35**

2. We assume that we would get revenue of 3500 THB if the campaign is success

In [273]:
tmp.shape[0]

2240

In [258]:
import random

chn = ['Online']*int(tmp.shape[0]*0.65) + ['Offline']*int(tmp.shape[0]*0.35)
random.shuffle(chn)

In [259]:
tmp['Channel'] = chn

### Focused on Online Channel 

In [268]:
threshold = [i/100 for i in range(0, 100, 2)]
cost = 1000
revenue = 3500

ts = []
ua_train_c = []
a_train_c = []
a_train_r = []
train_p = []

ua_test_c = []
a_test_c = []
a_test_r = []
test_p = []

for t in threshold:
    unaccept_train_cost = tmp[(tmp['type'] == 'y_train') & 
                              (tmp['Response'] == 0) & 
                              (tmp['predict_prob'] >= t) &
                              (tmp['Channel'] == "Online")].shape[0] * cost
    accept_train_cost = tmp[(tmp['type'] == 'y_train') & 
                            (tmp['Response'] == 1) & 
                            (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Online")].shape[0] * cost
    accept_train_rev = tmp[(tmp['type'] == 'y_train') & 
                           (tmp['Response'] == 1) & 
                           (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Online")].shape[0] * revenue
    train_profit = accept_train_rev - (accept_train_cost + unaccept_train_cost)
    
    unaccept_test_cost = tmp[(tmp['type'] == 'y_test') & 
                             (tmp['Response'] == 0) & 
                             (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Online")].shape[0] * cost
    accept_test_cost = tmp[(tmp['type'] == 'y_test') & 
                           (tmp['Response'] == 1) & 
                           (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Online")].shape[0] * cost
    accept_test_rev = tmp[(tmp['type'] == 'y_test') & 
                          (tmp['Response'] == 1) & 
                          (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Online")].shape[0] * revenue
    test_profit = accept_test_rev - (accept_test_cost + unaccept_test_cost)
    
    ts.append(t)
    ua_train_c.append(unaccept_train_cost)
    a_train_c.append(accept_train_cost)
    a_train_r.append(accept_train_rev)
    train_p.append(train_profit)
    
    ua_test_c.append(unaccept_test_cost)
    a_test_c.append(accept_test_cost)
    a_test_r.append(accept_test_rev)
    test_p.append(test_profit)
    
result = pd.DataFrame({
    'Threshold' : ts,
    'Train unaccepted cost' : ua_train_c,
    'Train accepted cost' : a_train_c,
    'Train accepted revenuc' : a_train_r,
    'Train Profit' : train_p,
    'Test unaccepted cost' : ua_test_c,
    'Test accepted cost' : a_test_c,
    'Test accepted revenuc' : a_test_r,
    'Test Profit' : test_p,
})

result.sort_values(by = 'Test Profit', ascending = False)

,Threshold,Train unaccepted cost,Train accepted cost,Train accepted revenuc,Train Profit,Test unaccepted cost,Test accepted cost,Test accepted revenuc,Test Profit
9,0.18,6000,155000,542500,381500,33000,34000,119000,52000
3,0.06,16000,155000,542500,371500,51000,41000,143500,51500
8,0.16,6000,155000,542500,381500,35000,34000,119000,50000
18,0.36,6000,155000,542500,381500,19000,27000,94500,48500
16,0.32,6000,155000,542500,381500,22000,28000,98000,48000
4,0.08,10000,155000,542500,377500,47000,38000,133000,48000
15,0.30,6000,155000,542500,381500,22000,28000,98000,48000
5,0.10,7000,155000,542500,380500,45000,37000,129500,47500
17,0.34,6000,155000,542500,381500,21000,27000,94500,46500
27,0.54,0,150000,525000,375000,9000,22000,77000,46000


In [297]:
t = tmp[(tmp['predict_prob'] >= 0.18) &
    (tmp['Channel'] == "Online")]
ev = round(t[t['Response'] == 1].shape[0]/t.shape[0]*100, 2)
cnt = tmp[tmp['Channel'] == 'Online'].shape[0]
print(f'Online count : {cnt}')
print(f'Count : {t.shape[0]}')
print(f'%Event Rate : {ev} %')

Online count : 1456
Count : 228
%Event Rate : 82.89 %


### Focused on Offline Channel

In [272]:
threshold = [i/100 for i in range(0, 100, 2)]
cost = 1500
revenue = 3500

ts = []
ua_train_c = []
a_train_c = []
a_train_r = []
train_p = []

ua_test_c = []
a_test_c = []
a_test_r = []
test_p = []

for t in threshold:
    unaccept_train_cost = tmp[(tmp['type'] == 'y_train') & 
                              (tmp['Response'] == 0) & 
                              (tmp['predict_prob'] >= t) &
                              (tmp['Channel'] == "Offline")].shape[0] * cost
    accept_train_cost = tmp[(tmp['type'] == 'y_train') & 
                            (tmp['Response'] == 1) & 
                            (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Offline")].shape[0] * cost
    accept_train_rev = tmp[(tmp['type'] == 'y_train') & 
                           (tmp['Response'] == 1) & 
                           (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Offline")].shape[0] * revenue
    train_profit = accept_train_rev - (accept_train_cost + unaccept_train_cost)
    
    unaccept_test_cost = tmp[(tmp['type'] == 'y_test') & 
                             (tmp['Response'] == 0) & 
                             (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Offline")].shape[0] * cost
    accept_test_cost = tmp[(tmp['type'] == 'y_test') & 
                           (tmp['Response'] == 1) & 
                           (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Offline")].shape[0] * cost
    accept_test_rev = tmp[(tmp['type'] == 'y_test') & 
                          (tmp['Response'] == 1) & 
                          (tmp['predict_prob'] >= t) &
                            (tmp['Channel'] == "Offline")].shape[0] * revenue
    test_profit = accept_test_rev - (accept_test_cost + unaccept_test_cost)
    
    ts.append(t)
    ua_train_c.append(unaccept_train_cost)
    a_train_c.append(accept_train_cost)
    a_train_r.append(accept_train_rev)
    train_p.append(train_profit)
    
    ua_test_c.append(unaccept_test_cost)
    a_test_c.append(accept_test_cost)
    a_test_r.append(accept_test_rev)
    test_p.append(test_profit)
    
result = pd.DataFrame({
    'Threshold' : ts,
    'Train unaccepted cost' : ua_train_c,
    'Train accepted cost' : a_train_c,
    'Train accepted revenuc' : a_train_r,
    'Train Profit' : train_p,
    'Test unaccepted cost' : ua_test_c,
    'Test accepted cost' : a_test_c,
    'Test accepted revenuc' : a_test_r,
    'Test Profit' : test_p,
})

result.sort_values(by = 'Test Profit', ascending = False)

,Threshold,Train unaccepted cost,Train accepted cost,Train accepted revenuc,Train Profit,Test unaccepted cost,Test accepted cost,Test accepted revenuc,Test Profit
28,0.56,1500,121500,283500,160500,9000,19500,45500,17000
29,0.58,0,121500,283500,162000,9000,19500,45500,17000
32,0.64,0,121500,283500,162000,7500,18000,42000,16500
31,0.62,0,121500,283500,162000,7500,18000,42000,16500
39,0.78,0,121500,283500,162000,6000,16500,38500,16000
38,0.76,0,121500,283500,162000,6000,16500,38500,16000
37,0.74,0,121500,283500,162000,6000,16500,38500,16000
36,0.72,0,121500,283500,162000,6000,16500,38500,16000
25,0.50,3000,123000,287000,161000,10500,19500,45500,15500
27,0.54,1500,121500,283500,160500,10500,19500,45500,15500


In [298]:
t = tmp[(tmp['predict_prob'] >= 0.56) &
    (tmp['Channel'] == "Offline")]
ev = round(t[t['Response'] == 1].shape[0]/t.shape[0]*100, 2)
cnt = tmp[tmp['Channel'] == 'Offline'].shape[0]
print(f'Offline count : {cnt}')
print(f'Count : {t.shape[0]}')
print(f'%Event Rate : {ev} %')

Offline count : 784
Count : 101
%Event Rate : 93.07 %
